# Ruteo

## Preparación de la Sesión

In [1]:
!git clone https://github.com/JuanCruzC97/ope3-logistica.git &> /dev/null

In [ ]:
cd ope3-logistica &> /dev/null

In [3]:
!pip install -e . &> /dev/null

## Módulos

In [6]:
import pandas as pd
import numpy as np

In [5]:
from logistica.ruteo import Ruteo
from logistica.utils import load_inputs, preparar_df_pedidos

In [19]:
from logistica import metaheuristicas as mh

# Carga de Datos

In [7]:
df_clientes = load_inputs("clientes")
df_pedidos = load_inputs("pedidos")
df_pedidos = pd.merge(left=df_pedidos, right=df_clientes, on="cliente", how="inner")
df_pedidos

,cliente,pedidos1,pedidos2,pedidos3,pedidos4,pedidos5,pedidos6,coord_x,coord_y
0,A,4,3,5,2,6,6,0.257571,1.803726
1,B,6,6,5,4,1,3,1.523313,2.102301
2,C,1,6,6,2,6,7,0.710910,2.629800
3,D,8,6,5,2,8,2,1.013940,2.149740
4,E,4,1,2,4,2,3,0.609480,0.666810
5,F,3,1,6,8,6,5,0.998640,1.451790
6,G,3,2,3,7,4,3,1.434510,0.018180
7,H,5,8,5,2,4,8,2.171880,0.764910
8,I,5,2,3,4,2,6,1.412280,0.863190
9,J,1,8,6,8,1,2,2.386260,1.387260


In [8]:
df_camiones = load_inputs("camiones")
df_camiones

,camion,carga_max,pedidos_max,dist_max
0,1,12,3,2
1,2,12,3,2
2,3,12,3,2
3,4,12,3,2
4,5,12,3,2
5,6,12,3,2


# Optimización 1 Pedido


In [27]:
#@title Seleccionar Pedido
pedido = 'pedidos3' #@param ["pedidos1", "pedidos2", "pedidos3", "pedidos4", "pedidos5", "pedidos6"]

In [28]:
df_pedido = preparar_df_pedidos(df_pedidos, pedido)
df_pedido

,cliente,pedidos,coord_x,coord_y
0,A,5,0.257571,1.803726
1,B,5,1.523313,2.102301
2,C,6,0.710910,2.629800
3,D,5,1.013940,2.149740
4,E,2,0.609480,0.666810
5,F,6,0.998640,1.451790
6,G,3,1.434510,0.018180
7,H,5,2.171880,0.764910
8,I,3,1.412280,0.863190
9,J,6,2.386260,1.387260


In [29]:
#@title Parámetros del Ruteo
costo_oportunidad = 3000 #@param {type:"number"}
presupuesto = 1220 #@param {type:"number"}

In [30]:
ruteo = Ruteo(df_camiones, df_pedido, costo_oportunidad=costo_oportunidad, presupuesto=presupuesto)

In [57]:
#@title Parámetros 
mode = 'random' #@param ["simple", "random"]
random_state = 45 #@param {type:"number"}

In [58]:
ruteo.get_solucion_inicial(mode=mode, random_state=random_state)

In [59]:
ruteo

--Ruteo--
Carga Total: 55tn
Costo Camiones: 61800$
Costo Oportunidad: 0$ 
Costo Total: 61800$ 
Costo Total por tn: 1123.64$/tn 
Ahorro: -7.9%

In [60]:
#@title Parámetros Metaheurística
temp_inicial = 500 #@param {type:"number"}
temp_final = 1 #@param {type:"number"}
k = 100 #@param {type:"number"}
iters = 20 #@param {type:"number"}
temp_mode = 'non_linear' #@param ["linear", "non_linear"]
p = 1 #@param {type:"number"}
random_state = 42 #@param {type:"number"}

In [61]:
best_sol, history = mh.sa(ruteo, 
                          t_inicial=temp_inicial, 
                          t_final=temp_final, 
                          k=k, 
                          iters=iters, 
                          temp_mode=temp_mode, 
                          prob=p, 
                          random_state=random_state)

100%|██████████| 1000/1000 [00:16<00:00, 59.36it/s]


In [62]:
best_sol

--Ruteo--
Carga Total: 55tn
Costo Camiones: 59000$
Costo Oportunidad: 0$ 
Costo Total: 59000$ 
Costo Total por tn: 1072.73$/tn 
Ahorro: -12.07%

In [63]:
best_sol.summary_camiones()

,Cliente 1,Cliente 2,Cliente 3,Costo,Carga,Costo_tn,Ahorro
Camion 1,L,A,I,11000,11,1000.0,-18.03
Camion 2,D,E,None,8400,7,1200.0,-1.64
Camion 3,H,F,None,11000,11,1000.0,-18.03
Camion 4,C,B,None,11000,11,1000.0,-18.03
Camion 5,G,None,None,5600,3,1866.67,53.01
Camion 6,J,K,None,12000,12,1000.0,-18.03
Costo Oportunidad,,,,0,0,,
Total,,,,59000,55,1072.73,-12.07


In [64]:
best_sol.summary_pedidos()

,Carga,Asignado,Camion
Pedido A,5,True,1
Pedido B,5,True,4
Pedido C,6,True,4
Pedido D,5,True,2
Pedido E,2,True,2
Pedido F,6,True,3
Pedido G,3,True,5
Pedido H,5,True,3
Pedido I,3,True,1
Pedido J,6,True,6
